In [ ]:
# %pip install findspark

In [ ]:
## Find spark absolute path and add .JAR file to pyspark jar folder 
## Since we wanna use spark we need jdbc(java data base connectivity) driver.
## In my case i use MS-SQL because of that i downloaded MS-SQL JDBC Driver from Microsoft.

## Microsoft Sql Server JDBC Driver
## https://docs.microsoft.com/en-us/sql/connect/jdbc/download-microsoft-jdbc-driver-for-sql-server?view=sql-server-ver17

## Consider picking that matches with your java installed on your machine. With pyspark or spark in general works great with java8 and java11.
## Since my choose was java11 i choose that version.

# import findspark
# findspark.init()
# findspark.find()

In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyodbc

In [2]:
spark = SparkSession.builder \
    .appName('Practice') \
    .master('local[*]') \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

In [ ]:
df_spark = spark.read.parquet('fhvhv_tripdata_2023-01.parquet', header=True)

In [ ]:
server = 'server'
database = 'database'
username = 'username'
password = 'password'
driver = 'com.microsoft.sqlserver.jdbc.SQLServerDriver' # its my driver
#port = port # Insert your pc port where your SQL SERVER is running

In [10]:
jdbc_url = f"jdbc:sqlserver://{server}:{port};databaseName={database};encrypt=false"
properties = {
    "user": username,
    "password": password,
    "driver": driver,
    "batchsize": "10000"
}

In [11]:
def create_database_if_not_exists(server, username, password, database):

    conn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};UID={username};PWD={password}')
    
    conn.autocommit = True
    
    cursor = conn.cursor()

    cursor.execute(f"""
    IF NOT EXISTS (SELECT * FROM sys.databases WHERE name = '{database}')
    BEGIN
        CREATE DATABASE {database}
    END
    """)
    
    cursor.close()
    conn.close()

In [12]:
create_database_if_not_exists(server, username, password, database)

In [13]:
df_spark = df_spark.repartition(100)

In [14]:
df_spark.write.jdbc(url=jdbc_url, table='fhvhv_tripdata', mode='append', properties=properties)